In [1]:
import numpy as np
import pandas as pd

# 1. Data cleaning

#### Anonimyze comment data and only keep number of characters

In [18]:
#comments are private, we used this function to mask them
def mask_comments(x):
    try:
        return '*'*len(x)
    except:
        return np.nan
comments['comment'] = comments['comment'].map(mask_comments)
comments.to_csv('data/co.csv')

## 1.1 Loading the data

In [2]:
comments = pd.read_csv('data/co.csv').drop('Unnamed: 0',axis=1).drop_duplicates()
comments.head(3)


,employee,companyAlias,commentId,comment,likes,dislikes,commentDate
0,307,56aec740f1ef260003e307d6,58d018d7e010990004e38070,**********************************************...,4.0,0.0,Mon Mar 20 19:00:17 CET 2017
1,382,56aec740f1ef260003e307d6,58d0179ae010990004e3806d,*****************************,1.0,2.0,Mon Mar 20 18:55:16 CET 2017
2,172,56aec740f1ef260003e307d6,58cff8cde010990004e37f6a,***************************,3.0,0.0,Mon Mar 20 16:44:02 CET 2017


In [7]:
lastpart = pd.read_csv('data/lastParticipationExists.csv').drop_duplicates()
lastpart.head(3)


,employee,companyAlias,numVotes,lastParticipationDate,stillExists
0,512,56aec740f1ef260003e307d6,4,Thu Feb 23 12:48:04 CET 2017,True
1,-2,56aec740f1ef260003e307d6,0,Wed Jan 18 14:00:55 CET 2017,False
2,2,56aec740f1ef260003e307d6,72,Fri Mar 17 01:00:00 CET 2017,True


In [8]:
votes = pd.read_csv('data/votes.csv',parse_dates=True,dayfirst=True).drop_duplicates()
votes.head(3)


,employee,companyAlias,voteDate,vote
0,31,56aec740f1ef260003e307d6,Mon Feb 01 01:00:00 CET 2016,4
1,33,56aec740f1ef260003e307d6,Mon Feb 01 01:00:00 CET 2016,4
2,79,56aec740f1ef260003e307d6,Mon Feb 01 01:00:00 CET 2016,4


In [9]:
inters = pd.read_csv('data/commentInteractions.csv').dropna(how='any').drop_duplicates()
inters.head(3)

/home/kalidus/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,employee,companyAlias,liked,disliked,commentId
0,307,56aec740f1ef260003e307d6,True,False,58d018d7e010990004e38070
1,36,56aec740f1ef260003e307d6,True,False,58d018d7e010990004e38070
2,276,56aec740f1ef260003e307d6,True,False,58d018d7e010990004e38070


## 1.2 Getting first dates and last dates of employee interactions

Convert strings representing dates to datetime objects.

In [11]:
lastpart['lastParticipationDate'] = lastpart['lastParticipationDate'].map(pd.to_datetime)
comments['commentDate'] = comments['commentDate'].map(pd.to_datetime)
votes['voteDate'] = votes['voteDate'].map(pd.to_datetime)

Get the first date an employee started using the app. I takes into account the first vote, the first comment written, and the first like/dislike, whatever it happened first.

In [15]:
min_date_comment = pd.DataFrame(comments.groupby(['companyAlias','employee'])['commentDate'].apply(np.min))
min_date_votes = pd.DataFrame(votes.groupby(['companyAlias','employee'])['voteDate'].apply(np.min))
min_date_part = pd.DataFrame(lastpart.groupby(['companyAlias','employee'])['lastParticipationDate'].min())
min_dates = pd.merge(min_date_comment,min_date_votes,right_index=True,left_index=True,how='outer')
min_dates = pd.merge(min_dates,min_date_part,right_index=True,left_index=True,how='outer')
first_dates = pd.DataFrame(min_dates.fillna(method='ffill',axis=1).fillna(method='bfill',axis=1).min(axis=1),columns=['first_date'])
first_dates.head(3)

first_date
companyAlias             employee                    
5370af43e4b0cff95558c12a -117     2016-03-04 09:55:34
                         -116     2016-09-12 11:53:45
                         -115     2016-09-12 11:54:00

Now we create a DataFrame containing the first date of app usage, the last date an employee used the app, and  a column that indicates if the user is still registered in the app.

In [20]:
last_dates = lastpart.set_index(['companyAlias','employee'])[['lastParticipationDate','stillExists']].copy()
dates = pd.merge(first_dates,last_dates,right_index=True,left_index=True,how='outer')
dates.ix[-5:-1].head()

first_date lastParticipationDate stillExists
companyAlias             employee                                             
58c6e15f32f72a00046f556c 58       2017-03-14   2017-03-14 23:13:43        True
                         59       2017-03-14   2017-03-14 23:13:43        True
                         60       2017-03-14   2017-03-20 14:29:20        True
                         61       2017-03-14   2017-03-14 23:13:43        True

## 1.3 Filtrering employees

- We delete employees churned before 30 june 2016 (arbtrary prediction date).
- We delete employees that started using the app after the prediction date.


In [21]:
prediction_date = '26 Dec 2016'
data_collection_ended = '20 March 2017'
last_dates['lastParticipationDate'].max()

Timestamp('2017-03-20 20:25:54')

In [22]:
c_first = dates['first_date']<=pd.to_datetime(prediction_date)
c_last = dates['lastParticipationDate']>pd.to_datetime(prediction_date)
dates[c_first&c_last]['stillExists'].value_counts()

True     1851
False     226
Name: stillExists, dtype: int64

## 1.4 Creating the churn variable

The churn feature will be 1 if a user churned after the prediction date. We also remove useres churned before prediction date.


In [31]:
users_index = dates[c_first&c_last].copy()

ce = users_index['stillExists'] == False
cd = users_index['lastParticipationDate']<pd.to_datetime(data_collection_ended)
#remove churned before pred. date
bad = users_index['lastParticipationDate']<pd.to_datetime(prediction_date)
users_index = users_index.loc[np.logical_not(bad)].copy()

users_index['churn'] = False
users_index.loc[ce&cd,'churn'] = True

users_index['churn'] = users_index['churn'].map(lambda x: int(x))
users_index.loc[np.logical_not(ce&cd)].shape,users_index[ce&cd].shape


((1868, 4), (209, 4))

## 1.5 Filter the original data to keep only valid ML employees

Keep only data generated by valid employees found in the users_index DataFrame.

In [51]:
alias_comp = dict([(x,i) for i,x in enumerate(np.unique(users_index.index.levels[0].values).tolist())])
users_index['comp_short'] = users_index.reset_index()['companyAlias'].map(lambda x: alias_comp[x]).values

valid_users = [(a,b) for a,b in users_index.reset_index()[['companyAlias','employee']].values.tolist()]

def filter_users(x):
    if (x['companyAlias'],x['employee']) in valid_users:
        return True
    else:
        return False

coms_ix = comments.apply(filter_users,axis=1)
lastp_ix = lastpart.apply(filter_users,axis=1)
votes_ix = votes.apply(filter_users,axis=1)
ints_ix = inters.apply(filter_users,axis=1)

inters_clean = inters[ints_ix].copy()
votes_clean = votes[votes_ix].copy()
coms_clean = comments[coms_ix].copy()
lastp_clean = lastpart[lastp_ix].copy()

## 1.6 Create UIDs for every employee

We map hash values to numeric values and create a unique identifier for each employee

In [54]:
def make_uid(x):
    df = x.reset_index().copy()
    return df['companyAlias'].map(lambda x: str(alias_comp[x])).values+'_'+df['employee'].map(str).values
users_index['uid'] = make_uid(users_index)
inters_clean['uid'] = make_uid(inters_clean)
inters_clean['comid'] = inters_clean['uid'].map(lambda x: x.split('_')[0]).values
votes_clean['uid'] = make_uid(votes_clean)
votes_clean['comid'] = votes_clean['uid'].map(lambda x: x.split('_')[0]).values
coms_clean['uid'] = make_uid(coms_clean)
coms_clean['comid'] = coms_clean['uid'].map(lambda x: x.split('_')[0]).values
lastp_clean['uid'] = make_uid(lastp_clean)
lastp_clean['comid'] = lastp_clean['uid'].map(lambda x: x.split('_')[0]).values
#in case you want to do inverse hashing
inv_comp = dict([(val,key) for key,val in alias_comp.items()])
def inv_company(x):
    c = x.split('_')[0]
    return inv_comp[int(c)]
    

## 1.7 Keep only data belonging to observation period

We make sure once more that we are only working with data belonging to observation period

In [55]:
#comments before 26 December 2016
coms_clean = coms_clean[coms_clean['commentDate']<=pd.to_datetime(prediction_date)].copy()
comment_ids = coms_clean['commentId'].unique().tolist()
valid_com = lambda x: x in comment_ids
inters_clean = inters_clean[inters_clean['commentId'].map(valid_com)].copy()

In [56]:

coms_clean = coms_clean[coms_clean['commentDate']<=pd.to_datetime(prediction_date)].copy()
comment_ids = coms_clean['commentId'].unique().tolist()
valid_com = lambda x: x in comment_ids
inters_clean = inters_clean[inters_clean['commentId'].map(valid_com)].copy()
votes_clean = votes_clean[votes_clean['voteDate']<=pd.to_datetime(prediction_date)].copy()

## 1.8 Delete duplicates

In [57]:
coms_clean = coms_clean.drop_duplicates()
votes_clean = votes_clean.drop_duplicates()
inters_clean = inters_clean.drop_duplicates()
lastp_clean = lastp_clean.drop_duplicates()
users_index = users_index.drop_duplicates()

# 2 Calculate employee features

We will only keep users with a minimum number of interactions recorded. So we now calculate the employee features to filter the data later.

## 2.1 vote features

In [40]:

vote_feats = votes_clean.groupby(['uid'])['vote'].agg({'votes_1': lambda x: len(x[x==1]),
                                                        'votes_2': lambda x: len(x[x==2]),
                                                        'votes_3': lambda x: len(x[x==3]),
                                                        'votes_4': lambda x: len(x[x==4]),
                                                        'votes_mean': lambda x: x.mean(),
                                                        'votes_std': lambda x: x.std(),
                                                       'votes_num': lambda x: len(x),
                                                       }).copy().fillna(0)

## 2.2 Comments features

In [41]:
likes_feats = coms_clean.groupby(['uid'])['likes'].agg({
                                          'likes_mean': lambda x: len(x),
                                          'likes_std': lambda x: x.std(),
                                          'likes_sum': lambda x: x.sum(),
                                                       })
dislikes_feats = coms_clean.groupby(['uid'])['dislikes'].agg({
                                          'dislikes_mean': lambda x: len(x),
                                          'dislikes_std': lambda x: x.std(),
                                          'dislikes_sum': lambda x: x.sum(),
                                                       })
coms_feats = coms_clean.dropna().groupby(['uid'])['comment'].agg({'com_num': lambda x: len(x),
                                                             'com_mean': lambda x: x.apply(len).mean(),
                                                             'com_std': lambda x: x.map(len).std(),
                                                             'com_sum': lambda x: x.map(len).sum()})

## 2.3 Merge all the features

In [42]:
features = pd.merge(vote_feats,likes_feats,left_index=True,right_index=True,how='outer')
features = pd.merge(features,dislikes_feats,left_index=True,right_index=True,how='outer')
features = pd.merge(features,coms_feats,left_index=True,right_index=True,how='outer')
features = pd.merge(features,users_index.set_index('uid')[['churn','comp_short']],left_index=True,right_index=True,how='left').fillna(0)#fill users with 0 comments
features.rename(columns={'comp_short':'comid'},inplace=True)
E_features = features.copy()
E_features.rename(columns=dict([(x,'E_'+str(x)) for x in E_features.columns]),inplace=True)


## 2.4 Only employees with more than 5 votes and interactions

This is the last step of our cleaning process. We will only keep users that meet the following criteria:

- **Has more than 5 votes recorded during observation period.**
- **The number of likes and dislikes recorded is at least 5.**

In [43]:
has_votes = E_features['E_votes_num']>=5
has_likes = (E_features['E_likes_sum']+E_features['E_dislikes_sum'])>=5
valid_uids = E_features[has_likes&has_votes].index.values.tolist()
E_features = E_features[has_likes&has_votes].copy()
has_votes = features['votes_num']>=5
has_likes = (features['likes_sum']+features['dislikes_sum'])>=5
features = features[has_likes&has_votes].copy()
valid_uid = lambda x: x in valid_uids

In [44]:
coms_clean = coms_clean[coms_clean['uid'].map(valid_uid)].drop_duplicates()
votes_clean = votes_clean[votes_clean['uid'].map(valid_uid)].drop_duplicates()
inters_clean = inters_clean[inters_clean['uid'].map(valid_uid)].drop_duplicates()
lastp_clean = lastp_clean[lastp_clean['uid'].map(valid_uid)].drop_duplicates()
users_index = users_index[users_index['uid'].map(valid_uid)].drop_duplicates()


## 2.5 Save the filtered data

Finally, we saved our machine learning data for further use.

In [3]:
inters_clean.to_csv('cleaned_data/interactions_ml.csv')
votes_clean.to_csv('cleaned_data/votes_ml.csv')
coms_clean.to_csv('cleaned_data/comments_ml.csv')
lastp_clean.to_csv('cleaned_data/lastpart_ml.csv')
users_index.to_csv('cleaned_data/target.csv')

NameError: name 'inters_clean' is not defined